In [7]:
#First create a data set for mean & standard deviation across each team each season
import pandas as pd
import numpy as np
import os

path = os.getcwd()

In [133]:
def aggregatePlayerStats(data, player_list):

    team_data = data[data['player'].isin(player_list)] #subset data in the player list
    sum_data = team_data[['fga', '3pa', '2pa', 'fta',
                       'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']].sum()
    avg_data = team_data[['2p%', '3p%', 'fg%', 'ft%', 'efg%', 'age', 'per', '3par',
                         'ftr', 'orb%', 'drb%', 'trb%', 'ast%', 'stl%', 'blk%', 'tov%',
                         'usg%', 'ows', 'dws', 'ws', 'ws/48', 'obpm', 'dbpm', 'bpm', 'vorp']].mean()
    final = pd.DataFrame(pd.concat([sum_data, avg_data], axis=0)).transpose()
    return final

In [121]:
# Function for calculating z scores
def getZScores(player_list, player_data, season_average_data):
    
    #season_average_data must be for the season in question... subset before hand\
    team_stats = aggregatePlayerStats(player_data, player_list)
    z_scores = list()
    #get mean/standard deviation for stats of specific season
    season_data = season_average_data.drop('season', axis=1) 
    #find the columns that intersect between two data sets to iterate over
    columns = team_stats.columns[team_stats.columns.isin(season_average_data.columns)]
    
    for stat in columns:
        if stat not in ['tov', 'pf']: #if not turnovers or personal fouls
            # z = (X - X_bar) / SD
            z = (team_stats[stat][0] - season_data.loc['mean', stat])/season_data.loc['sd', stat]
        else:
            # z = -(X - X_bar) / SD
            z = -(team_stats[stat][0] - season_data.loc['mean', stat])/season_data.loc['sd', stat]
            
        z_scores.append(z)
    
    # Put it all together in a single DataFrame
    z_scores = pd.DataFrame(z_scores, columns = ['z_score'], index = columns)
    z_scores = z_scores.sort_values('z_score', ascending = True)
    return z_scores

#Idea - should create a separate object that maps stat type to z-score goal for scalability - don't like
#having specific stats called out in function

In [10]:
def teamAssessment(data, bottom_n=3, top_n=3):
    
    strengths = list(data.tail(top_n).index)
    strengths.reverse() #reverse order
    weaknesses = list(data.head(bottom_n).index)
    
    results = {'weaknesses': "Your weakest {} areas are (in order): {}".format(str(bottom_n), str(weaknesses)),
            'strengths': "Your strongest {} areas are (in order): {}".format(str(top_n), str(strengths))}
    
    return(results)

#Next step: create dictionary that maps statistic abbreviation to human readable form, 
#use this in string generation

In [137]:
def assessPlayerSwaps(player_list, player_data, season_average_data, salary_data, salary_cap):
    
    potential_swaps = pd.DataFrame() #empty data frame
     
    og_team_stats = aggregatePlayerStats(player_data, player_list) #get the average stats 
    og_z_score = getZScores(player_list, player_data, season_average_data) #get z-score for each team stat
    og_cum_z_score = og_z_score.sum()[0] #get cumulative z score
    
    for player_new in player_data['player']:
        for player_og in player_list: #could add check here for whether player input is already on team
            new_player_list = [player_new if x == player_og else x for x in player_list] #create new player list

            if salary_cap == True:
                salary = salaryCap(new_player_list, salary_data, 80)

                if salary['over_cap'] == True:
                    continue

            new_team_stats = aggregatePlayerStats(player_data, new_player_list) #find stats for new team
            new_z_score = getZScores(new_player_list, player_data, season_average_data) #find z scores for new team
            new_cum_z_score = new_z_score.sum()[0] #find cumulative z score for new team

            cum_z_score_net_change = new_cum_z_score - og_cum_z_score #find net change in z score
            per_stat_diff = new_z_score - og_z_score #find per stat difference 
            stats_improved_count = len(per_stat_diff[per_stat_diff['z_score'] > 0]) #how many stats improved
            
            #Assign variables
            new_team_stats['player_og'] = player_og 
            new_team_stats['player_new'] = player_new
            new_team_stats['cum_z_score_og'] = og_cum_z_score
            new_team_stats['cum_z_score_new'] = new_cum_z_score
            new_team_stats['cum_z_score_net_change'] = cum_z_score_net_change
            new_team_stats['stats_improved_count'] = stats_improved_count
            
            #Concatentate with final df
            potential_swaps = pd.concat([potential_swaps, new_team_stats], axis=0)
            
    return(potential_swaps) #return results of all potential swaps

In [128]:
def recommendPlayer(player_list, player_swap_data):
    best_swap = player_swap_data[player_swap_data['cum_z_score_net_change'] ==
                                 max(player_swap_data['cum_z_score_net_change'])]
    
    string = "You should drop {} and add {}, he should help the team in {} statistical categories.".format(
        best_swap['player_og'][0], best_swap['player_new'][0], best_swap['stats_improved_count'][0])
    return(string)

#All sorts of potential improvements
#Salary cap limits 
#positional limits
#does this actually help the algorithmic prediction
#but for now, a POC that can work

In [ ]:
def salaryCap(player_list, salary_data, max_percentage):
    
    cap_total = 94.14 #hard coding for 2016
    cap_starters = cap_total * (max_percentage/100)
    salary = (salary_data[salary_data['player'].isin(player_list)]['salary'].sum(axis=0))/1000000
    
    if salary > cap_starters:
        return {'total_salary': salary, 'starter_cap': cap_starters, 'over_cap': True}
    else:
        return {'total_salary': salary, 'starter_cap': cap_starters, 'over_cap': False}

In [129]:
#Get player data
player_data = pd.read_csv(path+'/app/data/player_data_single_record.csv')
season_average_data = pd.read_csv(path+'/app/data/season_average_data.csv', index_col=0)
salary_data = pd.read_csv(path + '/processed_data/salaries_2017.csv')

In [130]:
#set player list + seeason
boo = ['Stephen Curry', 'Dwayne Wade', 'LeBron James', 'Chris Paul', 'Kyrie Irving']
season = 2016

In [131]:
player_data = player_data[player_data['season']==season] #subet by season
season_average_data = season_average_data[season_average_data['season']==season] #subset by season

In [126]:
#get the z score and a team assessment
z_score = getZScores(boo, player_data, season_average_data)
teamAssessment(z_score)


    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...     tov%  \
0  69.1  25.7  43.3  20.7  3.5  17.8  21.3  30.3  6.2  1.2  ...    13.35   

     usg%   ows   dws     ws  ws/48  obpm          dbpm   bpm   vorp  
0  28.825  8.45  2.75  11.25  0.221   6.8  8.326673e-17  6.75  5.425  

[1 rows x 38 columns]


{'strengths': "Your strongest 3 areas are (in order): ['ast%', 'ast', 'obpm']",
 'weaknesses': "Your weakest 3 areas are (in order): ['tov', 'blk%', 'blk']"}

In [138]:
#Figure out the ideal player swap
potential_swaps = assessPlayerSwaps(boo, player_data, season_average_data, salary_data, salary_cap=False)
recommendPlayer(boo, potential_swaps)

    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...     tov%  \
0  69.1  25.7  43.3  20.7  3.5  17.8  21.3  30.3  6.2  1.2  ...    13.35   

     usg%   ows   dws     ws  ws/48  obpm          dbpm   bpm   vorp  
0  28.825  8.45  2.75  11.25  0.221   6.8  8.326673e-17  6.75  5.425  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...   tov%  usg%  \
0  52.7  16.2  36.5  17.0  3.1  15.4  18.4  23.9  4.4  1.6  ...   14.2  25.7   

     ows    dws   ws   ws/48   obpm  dbpm    bpm  vorp  
0  6.225  1.825  8.1  0.1635  2.975   0.6  3.525  3.85  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...    tov%  \
0  71.0  26.2  44.8  21.6  3.9  19.1  22.9  30.5  6.2  1.8  ...   13.96   

    usg%   ows   dws   ws   ws/48  obpm  dbpm   bpm  vorp  
0  26.58  6.72  2.24  9.0  0.1766  3.94   0.4  4.28  4.32  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...      tov%  \
0  52.8  21.6

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...      tov%  \
0  57.9  15.7  42.1  17.4  3.8  17.2  21.0  24.6  4.7  1.2  ...    11.625   

     usg%    ows   dws     ws   ws/48   obpm   dbpm   bpm   vorp  
0  27.825  6.575  2.05  8.675  0.1935  4.475 -0.275  4.15  3.825  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...   tov%   usg%  \
0  76.2  25.7  50.4  22.0  4.6  20.9  25.5  31.2  6.5  1.4  ...   11.9  28.28   

   ows   dws    ws   ws/48  obpm  dbpm   bpm  vorp  
0  7.0  2.42  9.46  0.2006  5.14  -0.3  4.78   4.3  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...    tov%  \
0  58.0  21.1  36.9  14.8  3.3  13.6  16.9  22.5  5.3  0.8  ...   10.85   

    usg%  ows    dws   ws   ws/48   obpm   dbpm    bpm  vorp  
0  27.85  6.3  2.275  8.6  0.1955  4.725 -0.775  3.875  3.55  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...     tov%  

    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...    tov%  \
0  59.9  24.3  35.6  15.2  4.0  13.5  17.5  22.4  5.5  0.9  ...   10.55   

    usg%   ows    dws   ws   ws/48  obpm   dbpm    bpm  vorp  
0  25.55  6.05  2.025  8.1  0.1905  4.95 -0.875  4.025   3.6  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...     tov%  \
0  65.2  23.9  41.2  18.1  4.6  16.5  21.1  21.9  4.7  1.4  ...   11.075   

    usg%    ows    dws     ws    ws/48  obpm  dbpm    bpm  vorp  
0  26.95  6.525  2.125  8.675  0.17975   4.7 -0.75  3.925   4.1  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...   tov%   usg%  \
0  58.4  22.8  35.5  17.8  4.6  18.3  22.9  25.3  5.5  1.2  ...   12.0  25.35   

    ows  dws   ws    ws/48  obpm  dbpm  bpm  vorp  
0  6.65  2.4  9.1  0.20325  5.45   0.1  5.5   4.7  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...     tov%  \
0  55.4  15.8  39.5  17.1 

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...     tov%  \
0  61.7  23.0  38.5  17.7  3.5  16.8  20.3  24.5  5.2  1.6  ...    12.65   

     usg%  ows    dws      ws    ws/48   obpm  dbpm  bpm   vorp  
0  25.525  7.5  2.825  10.325  0.19675  5.175  0.15  5.3  4.725  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...      tov%  \
0  54.9  21.9  32.8  17.4  3.5  18.6  22.1  27.9  6.0  1.4  ...    13.575   

     usg%    ows  dws     ws    ws/48   obpm  dbpm    bpm   vorp  
0  23.925  7.625  3.1  10.75  0.22025  5.925   1.0  6.875  5.325  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...      tov%  \
0  56.7  18.0  38.5  17.5  3.9  18.0  21.9  24.7  5.6  2.0  ...    12.375   

     usg%   ows   dws    ws  ws/48   obpm   dbpm    bpm   vorp  
0  23.825  6.55  2.55  9.15  0.185  4.325  0.625  4.925  4.175  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...    tov%  \
0  73.1  27.8  45.2  22.3  4.6  20.1  24.7  30.8  6.4  1.3  ...   12.68   

    usg%   ows   dws   ws   ws/48  obpm  dbpm  bpm  vorp  
0  26.94  6.82  2.24  9.1  0.1948  5.28 -0.44  4.8  4.32  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...      tov%  \
0  54.9  23.2  31.7  15.1  3.3  12.8  16.1  22.1  5.2  0.7  ...    11.825   

     usg%    ows   dws    ws    ws/48  obpm  dbpm  bpm   vorp  
0  26.175  6.075  2.05  8.15  0.18825   4.9 -0.95  3.9  3.575  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...     tov%  \
0  60.2  22.8  37.3  18.0  3.9  15.8  19.7  21.6  4.4  1.2  ...    12.35   

     usg%   ows   dws     ws   ws/48  obpm   dbpm  bpm   vorp  
0  27.575  6.55  2.15  8.725  0.1775  4.65 -0.825  3.8  4.075  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...      tov%  \

...

    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...    tov%  \
0  53.9  15.7  38.1  16.9  3.5  15.7  19.2  24.5  4.5  1.4  ...   12.65   

     usg%  ows    dws    ws    ws/48   obpm  dbpm  bpm  vorp  
0  25.425  6.4  1.925  8.35  0.18725  4.025   0.3  4.3  3.85  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...    tov%  \
0  72.2  25.7  46.4  21.5  4.3  19.4  23.7  31.1  6.3  1.6  ...   12.72   

    usg%   ows   dws   ws   ws/48  obpm  dbpm  bpm  vorp  
0  26.36  6.86  2.32  9.2  0.1956  4.78  0.16  4.9  4.32  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...      tov%  \
0  54.0  21.1  32.9  14.3  3.0  12.1  15.1  22.4  5.1  1.0  ...    11.875   

    usg%    ows   dws     ws    ws/48   obpm  dbpm    bpm   vorp  
0  25.45  6.125  2.15  8.275  0.18925  4.275  -0.2  4.025  3.575  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...    tov%  \
0  59.3  20.7  38.5  

    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...     tov%  \
0  62.9  24.8  38.1  18.0  3.3  16.1  19.4  22.9  5.0  1.6  ...   11.975   

    usg%   ows    dws   ws   ws/48   obpm  dbpm  bpm  vorp  
0  26.45  6.95  2.625  9.6  0.1815  4.875 -0.05  4.8  4.45  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...   tov%   usg%  \
0  56.1  23.7  32.4  17.7  3.3  17.9  21.2  26.3  5.8  1.4  ...   12.9  24.85   

     ows  dws      ws  ws/48   obpm  dbpm    bpm  vorp  
0  7.075  2.9  10.025  0.205  5.625   0.8  6.375  5.05  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...      tov%  \
0  54.3  16.4  37.8  17.2  3.0  15.2  18.2  25.5  4.9  1.2  ...    15.825   

     usg%    ows  dws     ws   ws/48  obpm   dbpm  bpm   vorp  
0  26.075  6.075  1.9  8.025  0.1535   3.3  0.225  3.5  3.775  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...    tov%  \
0  72.6  26.4  46.1  

    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...    tov%  usg%  \
0  75.5  25.7  49.7  22.7  4.6  21.2  25.8  31.4  6.9  1.8  ...   12.92  27.0   

    ows  dws    ws   ws/48  obpm  dbpm   bpm  vorp  
0  7.04  2.5  9.58  0.1968  5.04  0.16  5.16   4.4  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...      tov%  \
0  57.3  21.1  36.2  15.5  3.3  13.9  17.2  22.7  5.7  1.2  ...    12.125   

    usg%   ows    dws    ws    ws/48  obpm  dbpm   bpm   vorp  
0  26.25  6.35  2.375  8.75  0.19075   4.6  -0.2  4.35  3.675  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...     tov%  \
0  62.6  20.7  41.8  18.4  3.9  16.9  20.8  22.2  4.9  1.7  ...    12.65   

    usg%    ows    dws     ws  ws/48  obpm   dbpm   bpm   vorp  
0  27.65  6.825  2.475  9.325   0.18  4.35 -0.075  4.25  4.175  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...      tov%  \
0  55.8  19.6  36.1

    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...   tov%  \
0  69.4  22.0  47.3  19.0  5.3  19.2  24.5  22.8  4.7  1.2  ...   12.0   

     usg%    ows  dws   ws   ws/48  obpm  dbpm    bpm  vorp  
0  30.025  6.675  2.7  9.4  0.1765   4.5 -0.45  4.025   4.1  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...     tov%  \
0  62.6  20.9  41.6  18.7  5.3  21.0  26.3  26.2  5.5  1.0  ...   12.925   

     usg%  ows    dws     ws  ws/48  obpm  dbpm  bpm  vorp  
0  28.425  6.8  2.975  9.825    0.2  5.25   0.4  5.6   4.7  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...    tov%  \
0  55.2  15.7  39.4  17.9  4.7  18.0  22.7  25.6  5.2  1.5  ...   14.95   

     usg%   ows  dws     ws    ws/48   obpm  dbpm  bpm  vorp  
0  25.125  6.85  2.4  9.275  0.20825  4.775  0.85  5.6  4.25  

[1 rows x 38 columns]
    fga   3pa   2pa   fta  orb   drb   trb   ast  stl  blk  ...    tov%  \
0  73.5  25.7  47.7  22.5  5.5 

'You should drop Kyrie Irving and add Russell Westbrook, he should help the team in 30 statistical categories.'

In [76]:
salary = pd.read_csv(path + '/processed_data/salaries_2017.csv')

In [112]:
def salaryCap(player_list, salary_data, max_percentage):
    
    cap_total = 94.14 #hard coding for 2016
    cap_starters = cap_total * (max_percentage/100)
    salary = (salary_data[salary_data['player'].isin(player_list)]['salary'].sum(axis=0))/1000000
    
    if salary > cap_starters:
        return {'total_salary': salary, 'starter_cap': cap_starters, 'over_cap': True}
    else:
        return {'total_salary': salary, 'starter_cap': cap_starters, 'over_cap': False}

In [134]:
weed = salaryCap(boo, salary_data, 80)

In [135]:
weed

{'over_cap': True, 'starter_cap': 75.312, 'total_salary': 83.5827}

In [111]:
salary_data

,player,salary
0,Al Horford,26540100.0
1,Amir Johnson,12000000.0
2,Avery Bradley,8269663.0
3,Demetrius Jackson,1450000.0
4,Gerald Green,1410598.0
5,Isaiah Thomas,6587132.0
6,Jae Crowder,6286408.0
7,James Young,1825200.0
8,Jaylen Brown,4743000.0
9,Jonas Jerebko,5000000.0
